In [24]:
from IPython.display import Audio, display
import numpy as np
import pandas as pd
from jiwer import wer, cer
from tqdm import tqdm

In [25]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"
import glob
import torch
import matplotlib.pyplot as pltm
sys.path.append("../")

In [26]:
longformdata = pd.read_csv("/app/dataset/metadata/annoated.csv",delimiter="	")

In [27]:
from configs.whisper_characterwise_pretrained_augs_small_openasr import Configs
CFG = Configs(longformdata.file.apply(lambda x: os.path.join("/app/dataset/examples",x)),longformdata.sentence)

inference mode is on


In [28]:
CFG.load_state_dict("/app/bengali-speech-recognition/workdir/whispersmall_characterlevel_finetuned_augmentations_openasr/bestmodel_wer.pkl")
CFG.model.cuda()
CFG.model.eval()
1

loading model checkpoint from epoch:  115000


1

In [33]:
def sliding_window(tensor, window_size, overlap):
    """
    Slices the input tensor with a sliding window of given size and overlap.
    
    Args:
        tensor (torch.Tensor): The input tensor of shape (points,).
        window_size (int): The size of the sliding window.
        overlap (int): The overlap between consecutive windows.
        
    Returns:
        torch.Tensor: A tensor containing the sliced windows stacked along the 0 dimension.
    """
    step = window_size - overlap
    slices = []
    
    for start in range(0, len(tensor) - window_size + 1, step):
        end = start + window_size
        slices.append(tensor[start:end])
    
    # Handle the final feature and pad it with zeros to match window_size
    if len(tensor) % step != 0:
        last_slice = tensor[-window_size:]
        slices.append(torch.nn.functional.pad(last_slice, (0, window_size - len(last_slice))))
    
    return torch.stack(slices, dim=0)
def sliding_window_spectro(tensor, window_size, overlap):
    """
    Slices the input tensor with a sliding window of given size and overlap.
    
    Args:
        tensor (torch.Tensor): The input tensor of shape (mels,points).
        window_size (int): The size of the sliding window.
        overlap (int): The overlap between consecutive windows.
        
    Returns:
        torch.Tensor: A tensor containing the sliced windows stacked along the 0 dimension.
    """
    step = window_size - overlap
    slices = []
    
    for start in range(0, tensor.shape[1] - window_size + 1, step):
        end = start + window_size
        slices.append(tensor[:,start:end])
    
    # Handle the final feature and pad it with zeros to match window_size
    if tensor.shape[1] % step != 0:
        last_slice = tensor[:,-window_size:]
        slices.append(torch.nn.functional.pad(last_slice, (0, window_size - last_slice.shape[1])))
    
    return torch.stack(slices, dim=0)

# ctc inference 

In [30]:
# all_truth= []
# all_preds = []
# OVERLAP=1
# for feature,truth in tqdm(CFG.inference_dataset):
#     with torch.no_grad():
#         feature = sliding_window(feature,CFG.MAX_AUDIO_LENGTH,OVERLAP).cuda()
#         preds = CFG.model(feature)
#         all_indices = torch.argmax(preds.detach().cpu(), dim=-1)
#     generated = []
#     for indices in all_indices:
#         indices = torch.unique_consecutive(indices, dim=-1)
#         indices = indices[indices != CFG.BLANK_TOKEN]
#         generated.extend(CFG.tokenizer.decode_torch_inference(indices))
#     prediction = "".join(generated)
#     all_truth.append(CFG.tokenizer.decode_torch_inference(truth))
#     all_preds.append(prediction)

# autoreg inference

In [36]:
all_truth= []
all_preds = []
OVERLAP=1
for feature,truth in tqdm(CFG.inference_dataset):
    with torch.no_grad():
        inputs = sliding_window_spectro(feature,1600,OVERLAP).cuda()
        batch_size = inputs.size(0)
        generated_tokens = torch.ones((batch_size, 1), dtype=torch.long, device="cuda") * CFG.START_TOKEN
        encoded_logits = CFG.model.encoder(inputs)
        eos_flags = torch.zeros(batch_size, dtype=torch.bool, device="cuda")

        for _ in range(CFG.MAX_PREDICTION_LENGTH):
            logits = CFG.model.decoder(generated_tokens, encoded_logits)
            next_token = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
            generated_tokens = torch.cat([generated_tokens, next_token], dim=1)

            # Update end-of-sequence flags
            eos_flags = eos_flags | (next_token.squeeze(-1) == CFG.END_TOKEN)

            # Stop condition: if all sequences in the batch have generated <eos>
            if eos_flags.all():
                break
    generated_tokens = generated_tokens[:, 1:]  # Remove the start token
    generated = []
    for gen in generated_tokens:
        end_pos = (gen == CFG.END_TOKEN).nonzero(as_tuple=True)[0]
        if len(end_pos) > 0:
            gen = gen[:end_pos[0]] 
        hypothesis = CFG.tokenizer.decode_torch_inference(gen)
        generated.append(hypothesis)
    prediction = "".join(generated)
    all_truth.append(CFG.tokenizer.decode_torch_inference(truth))
    all_preds.append(prediction)

100%|██████████| 17/17 [00:29<00:00,  1.76s/it]


In [37]:
longformdata["predictions"] = all_preds
longformdata["truth"] = all_truth
longformdata["wer"] = [wer(a,b) for a,b in zip(all_truth,all_preds)]
longformdata["cer"] = [cer(a,b) for a,b in zip(all_truth,all_preds)]


In [38]:
longformdata.wer.mean(),longformdata.cer.mean()

(0.9445320656091092, 0.7559441673709874)

In [11]:
longformdata.wer.mean(),longformdata.cer.mean()

(0.7978960718556445, 0.5293556438877308)